## Titanic survival

Neste notebook iremos analisar os dados do dataset Titanic para criar um modelo preditivo dos sobreviventes a partir de informações sobre os passageiros.  
O dataset utilizado pode ser conferido e baixado diretamente do Kaggle:
https://www.kaggle.com/c/titanic/

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import pickle

In [2]:
df = pd.read_csv('dataset/titanic_data.csv', skipinitialspace = True)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# Definindo as variáveis de input e a variável target
X = df.drop(['PassengerId', 'Name', 'Survived', 'Ticket', 'Cabin'], axis = 1)
Y = df[['Survived']]

In [6]:
# Realizando o split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 25)

In [7]:
# Verificando como as variáveis de teste e treino ficaram distribuídas
print("Original True : {0} ({1:0.2f}%)".format(len(df.loc[df['Survived'] == 1]), 
                                               (len(df.loc[df['Survived'] == 1])/len(df.index) * 100)))

print("Original False : {0} ({1:0.2f}%)".format(len(df.loc[df['Survived'] == 0]), 
                                               (len(df.loc[df['Survived'] == 0])/len(df.index) * 100)))
print("")
print("Training True : {0} ({1:0.2f}%)".format(len(Y_train[Y_train['Survived'] == 1]), 
                                               (len(Y_train[Y_train['Survived'] == 1])/len(Y_train) * 100)))

print("Training False : {0} ({1:0.2f}%)".format(len(Y_train[Y_train['Survived'] == 0]), 
                                               (len(Y_train[Y_train['Survived'] == 0])/len(Y_train) * 100)))
print("")
print("Test True : {0} ({1:0.2f}%)".format(len(Y_test[Y_test['Survived'] == 1]), 
                                               (len(Y_test[Y_test['Survived'] == 1])/len(Y_test) * 100)))

print("Test False : {0} ({1:0.2f}%)".format(len(Y_test[Y_test['Survived'] == 0]), 
                                               (len(Y_test[Y_test['Survived'] == 0])/len(Y_test) * 100)))

Original True : 342 (38.38%)
Original False : 549 (61.62%)

Training True : 239 (38.36%)
Training False : 384 (61.64%)

Test True : 103 (38.43%)
Test False : 165 (61.57%)


In [8]:
# Pipeline para tratar dados NaN e normalização
num_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')), # Como podemos observar, as distribuições não são gaussianas, logo, aplicamos a mediana
    ('scaler', StandardScaler())
])

# Pipeline para transformação de dados categóricos
cat_transformer = Pipeline(steps = [
    ('one-hot-encoder', OneHotEncoder())
])

# Compondo o pré-processamento dos dados
def preprocessor(numCol, catCol):
    preprocessing = ColumnTransformer(transformers = [
        ('num', num_transformer, numCol),
        ('cat', cat_transformer, catCol)
    ])
    return preprocessing

In [9]:
# Criando o modelo
preprocessing = preprocessor(['Age', 'SibSp', 'Parch', 'Fare'], ['Pclass', 'Sex', 'Embarked'])

model = Pipeline(steps = [
    ('preprocessing', preprocessing),
    ('randomforest', RandomForestClassifier(max_depth = 4, random_state = 42))
])

In [10]:
# Treinando o modelo
model.fit(X_train, Y_train.values.ravel())
train_score = model.score(X_train, Y_train.values.ravel())

In [11]:
# Testando o modelo
test_score = model.score(X_test, Y_test.values.ravel())
predict_test = model.predict(X_test)

In [12]:
# Outra maneira de calcular/mostrar o score
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_test, predict_test)))

Exatidão (Accuracy): 0.7985


In [13]:
# Preparando os dados para aplicar o teste chi2 a fim verificar as features mais relevantes para o modelo
sex_map = {'male': 0, 'female': 1}
embarked_map = {'C': 0, 'S': 1, 'Q': 2}

X_est = df.copy()
X_est['Sex'] = X_est['Sex'].map(sex_map)
X_est['Embarked'] = X_est['Embarked'].map(embarked_map)
X_est = X_est.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)
X_est.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    float64
dtypes: float64(3), int64(5)
memory usage: 55.8 KB


In [14]:
# Retirando as linhas com valores NaN para aplicarmos o método de análise estatística
X_est = X_est.dropna()
Y_est = X_est['Survived']
X_est = X_est.drop(['Survived'], axis = 1)
X_est.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    int64  
 2   Age       712 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  712 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 44.5 KB


In [15]:
Y_est = Y_est.reset_index(drop = True)
Y_est

0      0
1      1
2      1
3      1
4      0
      ..
707    0
708    0
709    1
710    1
711    0
Name: Survived, Length: 712, dtype: int64

In [16]:
# Aplicando teste chi2 para verificar as melhores features para o modelo
sel_feat = SelectKBest(score_func = chi2, k = 4).fit(X_est, Y_est)
print(sel_feat.scores_)
sel_bool = sel_feat.get_support()
# print(sel_bool)
# features = fit.transform(X_est)
# print(features[0:5,:])

[2.82432132e+01 1.30515207e+02 3.42460978e+01 2.88690988e-01
 1.08834998e+01 4.08167942e+03 6.02004416e+00]


In [17]:
# Criando as novas variáveis de treino e teste
X_train2 = X_train[X_train.columns[sel_bool]]
X_test2 = X_test[X_test.columns[sel_bool]]

In [18]:
preprocessing2 = preprocessor(['Age', 'Fare'], ['Pclass', 'Sex'])

model2 = Pipeline(steps = [
    ('preprocessing', preprocessing2),
    ('randomforest', RandomForestClassifier(max_depth = 4, random_state = 42))
])

In [19]:
# Treinando o modelo
model2.fit(X_train2, Y_train.values.ravel())
train_score2 = model2.score(X_train2, Y_train.values.ravel())

In [20]:
# Testando o modelo
test_score2 = model2.score(X_test2, Y_test.values.ravel())
predict_test2 = model2.predict(X_test2)

In [21]:
# Exibindo o score do novo modelo
print(f"Exatidão (Accuracy): {test_score2:0.4f}")

Exatidão (Accuracy): 0.8060


In [22]:
filename = 'model_titanic.sav'
pickle.dump(model, open(filename, 'wb'))